# WeatherPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
from pprint import pprint

# Import API key
from api_keys import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=15)
lngs = np.random.uniform(low=-180.000, high=180.000, size=15)
lat_lngs = zip(lats, lngs)


# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Create a DataFrame to hold the cities data
cities_df=pd.DataFrame(cities)

cities_df["Country"]=""
cities_df["Cloudiness"]=""
cities_df["Date"]=""
cities_df["Humidity"]=""
cities_df["Lat"]=""
cities_df["Lng"]=""
cities_df["Max Temp"]=""
cities_df["Wind Speed"]=""
cities_df=cities_df.rename(columns={0:"CityName"})
cities_df['CityName'] = (cities_df['CityName'].str.title())

cities_df

,CityName,Country,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Thompson,,,,,,,,
1,Albany,,,,,,,,
2,Flinders,,,,,,,,
3,Jamestown,,,,,,,,
4,Kiama,,,,,,,,
5,Kultuk,,,,,,,,
6,Bluff,,,,,,,,
7,Kapaa,,,,,,,,
8,Qaanaaq,,,,,,,,
9,Tiksi,,,,,,,,


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [3]:
# Save config information
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"
weatherdata = []
notfound = []

# Build query URL, get weather data for each city:
for index, row in cities_df.iterrows():
    city = row['CityName']
    query_url = f"{url}appid={api_key}&units={units}&q={city}"

    weather_responses = requests.get(query_url).json()
    weatherdata.append(weather_responses)
    
    try:
        cities_df.loc[index, 'Cloudiness'] = weatherdata[index]['clouds']['all']
        cities_df.loc[index, 'Country'] = weatherdata[index]['sys']['country']
        cities_df.loc[index, 'Date'] = weatherdata[index]['dt']
        cities_df.loc[index, 'Humidity'] = weatherdata[index]['main']['humidity']
        cities_df.loc[index, 'Lat'] = weatherdata[index]['coord']['lat']
        cities_df.loc[index, 'Lng'] = weatherdata[index]['coord']['lon']
        cities_df.loc[index, 'Max Temp'] = weatherdata[index]['main']['temp_max']
        cities_df.loc[index, 'Wind Speed'] = weatherdata[index]['wind']['speed']
        
    except (KeyError, IndexError):
        notfound.append(index)
        print(f"The city on row {index} is not found. Blank row")


In [ ]:
#print(f"The cities in rows {notfound} were not found and should be dropped")

In [4]:
#drop rows with incomplete data, print df
cities_df=cities_df.drop(notfound)
cities_df

,CityName,Country,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Thompson,CA,75,1570745495,47,55.74,-97.86,37.4,8.05
1,Albany,US,20,1570745495,42,42.65,-73.75,64.99,9.17
2,Flinders,AU,75,1570745496,87,-34.58,150.85,57,5.82
3,Jamestown,AU,0,1570745496,76,-33.21,138.6,46.51,14.06
4,Kiama,AU,75,1570745496,87,-34.67,150.86,57,5.82
5,Kultuk,RU,50,1570745497,84,51.73,103.71,12.96,1.59
6,Bluff,AU,21,1570745497,83,-23.58,149.07,64.69,7.97
7,Kapaa,US,1,1570745497,66,22.08,-159.32,86,5.82
8,Qaanaaq,GL,64,1570745497,61,77.48,-69.36,27.01,10.31
9,Tiksi,RU,94,1570745498,92,71.64,128.87,15.35,3.64


In [ ]:
cities_df.reset_index
cities_df

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [6]:
cities_df.to_csv("Mandell2WeatherData.csv", index=False, header=True)

In [7]:
cities_df.head(5)

,CityName,Country,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Thompson,CA,75,1570745495,47,55.74,-97.86,37.4,8.05
1,Albany,US,20,1570745495,42,42.65,-73.75,64.99,9.17
2,Flinders,AU,75,1570745496,87,-34.58,150.85,57,5.82
3,Jamestown,AU,0,1570745496,76,-33.21,138.6,46.51,14.06
4,Kiama,AU,75,1570745496,87,-34.67,150.86,57,5.82


### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

In [ ]:
latitude=cities_df.loc[:, "Lat"]
temp=cities_df.loc[:, "Max Temp"]
date=cities_df.iloc[1,3]
#date = date

plt.scatter(latitude, temp, marker="o", facecolors="red", edgecolors="black", alpha=.75)

plt.xlim(-100, 100)
plt.ylim(min(cities_df["Max Temp"])+5, max(cities_df["Max Temp"]+5))

plt.title(f"City Latitude vs Max Temperature ({date})")
plt.xlabel("Latitude of City")
plt.ylabel("Temperature (F)")
plt.show()


#### Latitude vs. Humidity Plot

#### Latitude vs. Cloudiness Plot

#### Latitude vs. Wind Speed Plot